In [72]:
import os
import shutil
from tempfile import mkdtemp, mkstemp

import h5py
import numpy as np

from deeprankcore.dataset import GraphDataset, GridDataset
from deeprankcore.domain import edgestorage as Efeat
from deeprankcore.domain import nodestorage as Nfeat
from deeprankcore.domain import targetstorage as targets
from deeprankcore.domain.aminoacidlist import (alanine, arginine, asparagine,
                                               glutamate, glycine, leucine,
                                               lysine, phenylalanine)
from deeprankcore.features import (components, conservation, contact,
                                   surfacearea)
from deeprankcore.query import (ProteinProteinInterfaceAtomicQuery,
                                ProteinProteinInterfaceResidueQuery,
                                QueryCollection,
                                SingleResidueVariantAtomicQuery,
                                SingleResidueVariantResidueQuery)
from deeprankcore.utils.grid import GridSettings, MapMethod

qc = QueryCollection()

for _ in range(5): 
    qc.add(ProteinProteinInterfaceResidueQuery(
        "tests/data/pdb/3C8P/3C8P.pdb",
        "A",
        "B",
        {
            "A": "tests/data/pssm/3C8P/3C8P.A.pdb.pssm",
            "B": "tests/data/pssm/3C8P/3C8P.B.pdb.pssm",
        },
        targets={targets.BINARY: 1.2},
    ))

augmentation_count = 3
grid_settings = GridSettings([20, 20, 20], [1.0, 1.0, 1.0])

expected_entry_count = (augmentation_count + 1) * len(qc)

try:
    qc.process(f"new_test",
                feature_modules=None,
                grid_settings=grid_settings,
                grid_map_method=MapMethod.GAUSSIAN,
                grid_augmentation_count=augmentation_count)

    hdf5_path = f"new_test.hdf5"

    with h5py.File(hdf5_path, 'r') as f5:
        entry_names = list(f5.keys())

    dataset = GridDataset(hdf5_path)

SyntaxError: expected ':' (2480970219.py, line 26)